# getting top cities information from wiki and their location

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
html = requests.get(url).content
df_list = pd.read_html(html)
df=df_list[4]
df.head()

,2019rank,City,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York[d],New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston[3],Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


In [3]:
df=df[['City','2019estimate','2016 population density','Location']]
location=[]
for i in range(df.shape[0]):
    tmp_list=[]
    tmp=df.loc[i]['City']
    if '[' in tmp:
        df.at[i, 'City']=tmp.split('[')[0]
    
    tmp=df.loc[i]['2019estimate']
    df.at[i,'2019estimate']=float(tmp)
        
    tmp=df.loc[i]['2016 population density']
    tmp=tmp.split('/')[0]
    df.at[i,'2016 population density']=float(tmp.replace(",",""))
    
    tmp=df.loc[i]['Location']
    lati=tmp.split(' ')[-2]
    lati=lati.split('°')[0]
    tmp_list.append(float(lati[1:]))                    
    longi=tmp.split(' ')[-1]
    tmp_list.append(float(longi.split('°')[0]))   
    location.append(tmp_list)
    

df.head()

,City,2019estimate,2016 population density,Location
0,New York,8336817,28317,40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,Los Angeles,3979576,8484,34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,Chicago,2693976,11900,41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,Houston,2320268,3613,29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,Phoenix,1680992,3120,33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


In [4]:
from pandas import DataFrame

df_location = DataFrame(location,columns=['lantitude','longitude'])
df_combine = pd.concat([df, df_location], axis=1)
df_combine.drop(['Location'], axis=1, inplace=True)
df_combine.to_csv(r'city_data.csv', index = False)
df_combine.head()

,City,2019estimate,2016 population density,lantitude,longitude
0,New York,8336817,28317,40.6635,73.9387
1,Los Angeles,3979576,8484,34.0194,118.4108
2,Chicago,2693976,11900,41.8376,87.6818
3,Houston,2320268,3613,29.7866,95.3909
4,Phoenix,1680992,3120,33.5722,112.0901


# start using foursquare to get info for each city

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

CLIENT_ID = '5BN5APUDLOMB2J35RJU0ZP2XAZBKEO03PDKGQ1VPNZ0PQS1M' # your Foursquare ID
CLIENT_SECRET = 'NGULJIIX5W5JBFM4NAUVK41KNVAM0FSH5STKQM24VBCNKDPE' # your Foursquare Secret
VERSION = '20180605'

print('Libraries imported.')

Libraries imported.


In [20]:
radius=3000
LIMIT=200
search_query='restaurant'
latitude=df_combine.loc[0]['lantitude']
longitude=-df_combine.loc[0]['longitude']

In [60]:
def getVenues(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT=200

        search_query='restaurant'
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["venues"]
        
        
        
        for i in range(len(results)):
            re=results[i]
            if re['categories'] != []:
                tmp=[]
                tmp.append(name)
                tmp.append(lat)
                tmp.append(lng)
                tmp.append(re['name'])  
                tmp.append(re['categories'][0]['name'])   
                venues_list.append(tmp)  
            
        
        # return only relevant information for each nearby venue
       
    nearby_venues = pd.DataFrame(venues_list,columns = ['City', 
                  'lantitude', 
                  'longitude', 
                  'Venue', 
                  'Venue Category'])
    
    return(nearby_venues)

In [62]:
us_city_restaurant= getVenues(names=df_combine['City'],
                                   latitudes=df_combine['lantitude'],
                                   longitudes=-df_combine['longitude']
                                  )
us_city_restaurant.head()

New York
Los Angeles
Chicago
Houston
Phoenix
Philadelphia
San Antonio
San Diego
Dallas
San Jose
Austin
Jacksonville
Fort Worth
Columbus
Charlotte
San Francisco
Indianapolis
Seattle
Denver
Washington
Boston
El Paso
Nashville
Detroit
Oklahoma City
Portland
Las Vegas
Memphis
Louisville
Baltimore
Milwaukee
Albuquerque
Tucson
Fresno
Mesa
Sacramento
Atlanta
Kansas City
Colorado Springs
Omaha
Raleigh
Miami
Long Beach
Virginia Beach
Oakland
Minneapolis
Tulsa
Tampa
Arlington
New Orleans
Wichita
Bakersfield
Cleveland
Aurora
Anaheim
Honolulu
Santa Ana
Riverside
Corpus Christi
Lexington
Henderson
Stockton
Saint Paul
Cincinnati
St. Louis
Pittsburgh
Greensboro
Lincoln
Anchorage
Plano
Orlando
Irvine
Newark
Durham
Chula Vista
Toledo
Fort Wayne
St. Petersburg
Laredo
Jersey City
Chandler
Madison
Lubbock
Scottsdale
Reno
Buffalo
Gilbert
Glendale
North Las Vegas
Winston–Salem
Chesapeake
Norfolk
Fremont
Garland
Irving
Hialeah
Richmond
Boise
Spokane
Baton Rouge
Tacoma
San Bernardino
Modesto
Fontana
Des Moine

,City,lantitude,longitude,Venue,Venue Category
0,New York,40.6635,-73.9387,Wing Hing Restaurant,Chinese Restaurant
1,New York,40.6635,-73.9387,New Garden Restaurant,Chinese Restaurant
2,New York,40.6635,-73.9387,Vee's Jamaican Restaurant,Caribbean Restaurant
3,New York,40.6635,-73.9387,New Combination Restaurant,African Restaurant
4,New York,40.6635,-73.9387,C & J Jamaican Restaurant,Caribbean Restaurant


In [63]:
us_city_restaurant.to_csv(r'restaurant_data.csv', index = False)


In [2]:
import pandas as pd

In [3]:
us_city_restaurant=pd.read_csv('restaurant_data.csv')
us_city_restaurant.head()

,City,lantitude,longitude,Venue,Venue Category
0,New York,40.6635,-73.9387,Wing Hing Restaurant,Chinese Restaurant
1,New York,40.6635,-73.9387,New Garden Restaurant,Chinese Restaurant
2,New York,40.6635,-73.9387,Vee's Jamaican Restaurant,Caribbean Restaurant
3,New York,40.6635,-73.9387,New Combination Restaurant,African Restaurant
4,New York,40.6635,-73.9387,C & J Jamaican Restaurant,Caribbean Restaurant


 # from here begin to get frequency of restaurant category in each cicty

In [4]:
restaurant_onehot = pd.get_dummies(us_city_restaurant[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
restaurant_onehot['City'] = us_city_restaurant['City'] 

# move neighborhood column to the first column
fixed_columns = [restaurant_onehot.columns[-1]] + list(restaurant_onehot.columns[:-1])
restaurant_onehot = restaurant_onehot[fixed_columns]

restaurant_grouped = restaurant_onehot.groupby('City').mean().reset_index()
restaurant_grouped.head()

,City,Yemeni Restaurant,ATM,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Vietnamese Restaurant,Warehouse,Warehouse Store,Whisky Bar,Wine Bar,Winery,Wings Joint
0,Abilene,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
1,Akron,0.0,0.0,0.0,0.0,0.0,0.117647,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
2,Albuquerque,0.0,0.0,0.0,0.0,0.0,0.131579,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
3,Alexandria,0.0,0.0,0.0,0.0,0.0,0.073171,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.02439,0.0,0.0,0.0,0.0
4,Allen,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0


In [6]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Res'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Res'.format(ind+1))

# create a new dataframe
restaurant_sorted = pd.DataFrame(columns=columns)
restaurant_sorted['City'] = restaurant_grouped['City']

for ind in np.arange(restaurant_grouped.shape[0]):
    restaurant_sorted.iloc[ind, 1:] = return_most_common_venues(restaurant_grouped.iloc[ind, :], num_top_venues)

restaurant_sorted.head()

,City,1st Most Common Res,2nd Most Common Res,3rd Most Common Res,4th Most Common Res,5th Most Common Res
0,Abilene,Chinese Restaurant,Mexican Restaurant,Food,Ice Cream Shop,American Restaurant
1,Akron,Food,Italian Restaurant,American Restaurant,Event Space,Kitchen Supply Store
2,Albuquerque,Food,Mexican Restaurant,American Restaurant,Restaurant,Chinese Restaurant
3,Alexandria,Restaurant,American Restaurant,Latin American Restaurant,Food,Office
4,Allen,Food,Mexican Restaurant,Italian Restaurant,Restaurant,Sports Bar


# we will combine frequency of restanrant and what df containes population in step 1 to get final data

In [33]:
df=pd.read_csv('city_data.csv')

City                        object
2019estimate                 int64
2016 population density    float64
lantitude                  float64
longitude                  float64
dtype: object

In [36]:
#normalize df in step 1
df['2019estimate'] = df['2019estimate'].astype(float)
dens_max=max(df['2016 population density'])
dens_min=min(df['2016 population density'])
pop_max=max(df['2019estimate'])
pop_min=min(df['2019estimate'])

for i in range(df.shape[0]):
    tmp=df.loc[i]['2019estimate']
    df.at[i,'2019estimate']=(float(tmp))/float(pop_max)
    tmp=df.loc[i]['2016 population density']
    df.at[i,'2016 population density']=(float(tmp)-dens_min)/(dens_max-dens_min)
    

df.drop(['lantitude'], axis=1, inplace=True)
df.drop(['longitude'], axis=1, inplace=True)

df.head()


,City,2019estimate,2016 population density
0,New York,1.000000,1.000000
1,Los Angeles,0.477350,0.295253
2,Chicago,0.323142,0.416637
3,Houston,0.278316,0.122166
4,Phoenix,0.201635,0.104648


In [37]:
data_df = pd.merge(df, restaurant_grouped, how='left', on='City')
data_df.head()

,City,2019estimate,2016 population density,Yemeni Restaurant,ATM,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Vietnamese Restaurant,Warehouse,Warehouse Store,Whisky Bar,Wine Bar,Winery,Wings Joint
0,New York,1.000000,1.000000,0.0,0.0,0.0,0.0,0.041667,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,Los Angeles,0.477350,0.295253,0.0,0.0,0.0,0.0,0.000000,0.040816,0.000000,...,0.020408,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,Chicago,0.323142,0.416637,0.0,0.0,0.0,0.0,0.000000,0.040816,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,Houston,0.278316,0.122166,0.0,0.0,0.0,0.0,0.000000,0.023256,0.023256,...,0.000000,0.0,0.0,0.023256,0.0,0.0,0.0,0.023256,0.0,0.0
4,Phoenix,0.201635,0.104648,0.0,0.0,0.0,0.0,0.000000,0.086957,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
